In [1]:
import sys
import numpy as np
from functools import reduce
from pyscf.lib import logger
from ad_afqmc.lno.cc import LNOCCSD
from ad_afqmc import lno_ccsd
from pyscf import gto, scf, mp, cc, fci

log = logger.Logger(sys.stdout, 6)

a = 0.9
nH = 6
atoms = ""
for i in range(nH):
    atoms += f"H {i*a} 0 0 \n"

mol = gto.M(atom=atoms, basis="sto6g", verbose=4)
mf = scf.RHF(mol).density_fit()
mf.kernel()

frozen = 0
mmp = mp.MP2(mf,frozen=frozen)
mmp.kernel()[0]

# cc
mycc = cc.CCSD(mf)
mycc.kernel()
et = mycc.ccsd_t()

# fci
cisolver = fci.FCI(mf)
fci_ene, fci_vec = cisolver.kernel()

print(f'rhf energy is {mf.e_tot}')
print(f"ccsd energy is {mycc.e_tot}")
print(f"ccsd_t energy is {mycc.e_tot+et}")
print(f"fci_ene: {fci_ene}", flush=True)

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64
System: uname_result(system='Linux', node='yichi-thinkpad', release='4.4.0-26100-Microsoft', version='#1882-Microsoft Fri Jan 01 08:00:00 PST 2016', machine='x86_64')  Threads 12
Python 3.10.16 | packaged by conda-forge | (main, Dec  5 2024, 14:16:10) [GCC 13.3.0]
numpy 1.24.3  scipy 1.14.1  h5py 3.12.1
Date: Sun May  4 22:13:11 2025
PySCF version 2.8.0
PySCF path  /home/yichi/research/software/lno_pyscf
GIT HEAD (branch master) ef75f4190e4de208685670651dc6c467f72b6794

[ENV] PYSCF_EXT_PATH /home/yichi/research/software/pyscf
[CONFIG] conf_file None
[INPUT] verbose = 4
[INPUT] num. atoms = 6
[INPUT] num. electrons = 6
[INPUT] charge = 0
[INPUT] spin (= nelec alpha-beta = 2S) = 0
[INPUT] symmetry False subgroup None
[INPUT] Mole.unit = angstrom
[INPUT] Symbol           X                Y                Z      unit          X                Y                Z       unit  Magmom
[INPUT]  1 H      0.

In [2]:
thresh = 1e-4
lno_cc = LNOCCSD(mf, thresh=thresh, frozen=frozen).set(verbose=5)
lno_cc.thresh_occ = thresh
lno_cc.thresh_vir = thresh
lno_cc.lo_type = 'pm'
lno_cc.no_type = 'cim'
lno_cc.frag_lolist = '1o'
lno_cc.ccsd_t = True
lno_cc.force_outcore_ao2mo = True

orbloc = lno_cc.orbloc
frag_atmlist = lno_cc.frag_atmlist
frag_lolist = lno_cc.frag_lolist
s1e = lno_cc._scf.get_ovlp()

_fdot = np.dot
fdot = lambda *args: reduce(_fdot, args)
from ad_afqmc.lno.base import lno

# NO type
no_type = 'ie'
lo_type = 'pm'
frag_lolist = '1o'
log.info('no_type = %s', no_type)

# LO construction
orbloc = lno_cc.get_lo(lo_type=lo_type) # localized active occ orbitals
orbactocc = lno_cc.split_mo()[1] # non-localized active occ
m = fdot(orbloc.T, s1e, orbactocc)
lospanerr = abs(fdot(m.T, m) - np.eye(m.shape[1])).max()
if lospanerr > 1e-10:
    log.error('LOs do not fully span the occupied space! '
                'Max|<occ|LO><LO|occ>| = %e', lospanerr)
    raise RuntimeError

# check 2: Span(LO) == Span(occ)
occspanerr = abs(fdot(m, m.T) - np.eye(m.shape[0])).max()
if occspanerr < 1e-10:
    log.info('LOs span exactly the occupied space.')
    if no_type not in ['ir','ie']:
        log.error('"no_type" must be "ir" or "ie".')
        raise ValueError
else:
    log.info('LOs span occupied space plus some virtual space.')

# LO assignment to fragments

if frag_lolist == '1o':
    log.info('Using single-LO fragment') # this is what we use, every active local occ stands for a fragment
    frag_lolist = [[i] for i in range(orbloc.shape[1])]
nfrag = len(frag_lolist)

frag_nonvlist = lno_cc.frag_nonvlist

canonicalize = False
if not (no_type[0] in 'rei' and no_type[1] in 'rei'):
    log.warn('Input no_type "%s" is invalid.', no_type)
    raise ValueError

if frag_nonvlist is None: frag_nonvlist = [[None,None]] * nfrag

eris = lno_cc.ao2mo()
frag_res = [None] * nfrag

# dump info
log.info('nfrag = %d  nlo = %d', nfrag, orbloc.shape[1])
#log.info('frag_atmlist = %s', frag_atmlist)
log.info('frag_lolist = %s', frag_lolist)
#log.info('frag_nonvlist = %s', frag_nonvlist)

no_type = ie
lo_type = pm
LOs span exactly the occupied space.
Using single-LO fragment
ao2mo est mem= 0.01 MB  avail mem= 3802.81 MB
Lov is saved to /tmp/tkjndfth
    CPU time for transforming DF-MP2 integrals      0.00 sec, wall time      0.00 sec
    CPU time for Integral xform         0.02 sec, wall time      0.01 sec
nfrag = 3  nlo = 3
frag_lolist = [[0], [1], [2]]


In [3]:
from ad_afqmc.lno.afqmc import LNOAFQMC
lno_qmc = LNOAFQMC(mf, thresh=thresh).set(verbose=5)
lno_qmc.thresh_occ = thresh
lno_qmc.thresh_vir = thresh
lno_qmc.nblocks = 10
lno_qmc.nwalk_per_proc = 10
lno_qmc.nproc = 8
lno_qmc.lo_type = 'pm'
lno_qmc.no_type = 'cim'
lno_qmc.frag_lolist = '1o'
lno_qmc.chol_cut = 1e-6

from ad_afqmc import config
config.setup_jax()
MPI = config.setup_comm()
comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

from functools import partial

from jax import numpy as jnp

from ad_afqmc import run_afqmc

print = partial(print, flush=True)

mo_file = run_afqmc.mo_file
amp_file = run_afqmc.amp_file
chol_file = run_afqmc.chol_file

options = {'n_eql': 2,
            'n_ene_blocks': 1,
            'n_sr_blocks': 10,
            'n_blocks': 10,
            'n_walkers': 10,
            'seed': 98,
            'walker_type': 'rhf',
            'trial': 'cisd',
            'dt':0.01,
            'ad_mode':None,
            }

from jax import random
from ad_afqmc import wavefunctions

# Hostname: yichi-thinkpad
# System Type: Linux
# Machine Type: x86_64
# Processor: x86_64


In [4]:
lno_mf = lno_cc._scf
ecorr_ccsd = np.empty(nfrag)
hf_elec = np.empty(nfrag)
block_orb_energy = np.empty(nfrag)
frozen_mask = lno_cc.get_frozen_mask()
thresh_pno = [lno_cc.thresh_occ, lno_cc.thresh_vir]
for ifrag in range(nfrag):
    print(ifrag)
    if(len(lno_cc.runfrags)>0):
        if(ifrag not in lno_cc.runfrags):frag_res[ifrag] = (0,0,0)
    fraglo = frag_lolist[ifrag]
    orbfragloc = orbloc[:,fraglo] # the specific local active occ
    frag_target_nocc, frag_target_nvir = frag_nonvlist[ifrag]
    THRESH_INTERNAL = 1e-10
    frzfrag, orbfrag, can_orbfrag = lno.make_fpno1(lno_cc, eris, orbfragloc, no_type,
                                                THRESH_INTERNAL, thresh_pno,
                                                frozen_mask=frozen_mask,
                                                frag_target_nocc=frag_target_nocc,
                                                frag_target_nvir=frag_target_nvir,
                                                canonicalize=False)
    # print(orbfragloc)
    # print(orbfrag)
    # print(fdot(orbfragloc.T, s1e, orbfrag))
    print(frzfrag)

    ecorr_ccsd[ifrag],_,t1,t2 = lno_ccsd.cc_impurity_solve(mf,orbfrag,orbfragloc,frozen=frzfrag,eris=eris,log=log)
    hf_elec[ifrag] = lno_ccsd.e_elec_frg(mf,orbfragloc,orbfrag,frozen=frzfrag)
    print('ecorr_ccsd', ecorr_ccsd[ifrag])
    print('hf_elec', hf_elec[ifrag])

    frozen=frzfrag
    # can_orbfrag = can_orbfrag

    maskocc = mf.mo_occ>1e-10
    nmo = mf.mo_occ.size

    # Convert frozen to 0 bc PySCF solvers do not support frozen=None or empty list
    if frozen is None:
        frozen = 0
    elif isinstance(frozen, (list,tuple,np.ndarray)) and len(frozen) == 0:
        frozen = 0

    if isinstance(frozen, (int,np.integer)):
        maskact = np.hstack([np.zeros(frozen,dtype=bool),
                                np.ones(nmo-frozen,dtype=bool)])
    elif isinstance(frozen, (list,tuple,np.ndarray)):
        maskact = np.array([i not in frozen for i in range(nmo)])
    else:
        raise RuntimeError

    orbfrzocc = orbfrag[:,~maskact& maskocc]
    orbactocc = orbfrag[:, maskact& maskocc]
    orbactvir = orbfrag[:, maskact&~maskocc]
    orbfrzvir = orbfrag[:,~maskact&~maskocc]
    nfrzocc, nactocc, nactvir, nfrzvir = [orb.shape[1]
                                            for orb in [orbfrzocc,orbactocc,
                                                        orbactvir,orbfrzvir]]
    s1e = mf.get_ovlp() if eris is None else eris.s1e
    prjlo = fdot(orbfragloc.T, s1e, orbactocc) ### overlap between the lo and each active occ in its fragment
    print(prjlo)

    lno_ccsd.prep_lno_amp_chol_file(mycc,orbfrag,norb_act=(nactocc+nactvir),nelec_act=nactocc*2,norb_frozen=frozen,
                t1=t1,t2=t2,mo_file="mo_test",amp_file="amp_test",chol_file="chol_test")
    
    ham_data, ham, prop, trial, wave_data, sampler, observable, options, _ = \
        (lno_ccsd.prep_lnoccsd_afqmc(options,prjlo=prjlo,mo_file="mo_test.npz",amp_file="amp_test.npz",chol_file="chol_test"))
    
    seed = options["seed"]
    propagator = prop
    init_walkers = None
    ham_data = wavefunctions.rhf(trial.norb, trial.nelec, n_batch=trial.n_batch)._build_measurement_intermediates(ham_data, wave_data)
    ham_data = ham.build_measurement_intermediates(ham_data, trial, wave_data)
    ham_data = ham.build_propagation_intermediates(ham_data, propagator, trial, wave_data)

    prop_data = propagator.init_prop_data(trial, wave_data, ham_data, init_walkers)
    if jnp.abs(jnp.sum(prop_data["overlaps"])) < 1.0e-6:
        raise ValueError(
            "Initial overlaps are zero. Pass walkers with non-zero overlap."
        )
    prop_data["key"] = random.PRNGKey(seed + rank)

    prop_data["overlaps"] = trial.calc_overlap(prop_data["walkers"], wave_data)
    prop_data["n_killed_walkers"] = 0
    prop_data["pop_control_ene_shift"] = prop_data["e_estimate"]
    #e_elec = lno_ccsd.e_elec_frg(mf,orbfragloc,orbfrag,frzfrag)
    block_orb_energy[ifrag], prop_data = \
        lno_ccsd.propagate_phaseless_orb(ham_data, prop, prop_data, trial, wave_data,hf_elec[ifrag])
    print('orbital energy: ',block_orb_energy[ifrag])
    


print(sum(ecorr_ccsd))
print(sum(hf_elec))

0
[]
    impsol:  1 LOs  6/6 MOs  3 occ  3 vir
    CPU time for imp sol - eri          0.08 sec, wall time      0.01 sec
    CPU time for imp sol - mp2 amp      0.00 sec, wall time      0.01 sec
    CPU time for imp sol - mp2 ene      0.03 sec, wall time      0.00 sec
    CPU time for imp sol - cc  amp      0.20 sec, wall time      0.15 sec
    CPU time for imp sol - cc  ene      0.00 sec, wall time      0.00 sec
ecorr_ccsd -0.027762172214545248
hf_elec -3.0126926975873216
[[ 1.66533454e-16 -1.38777878e-16 -1.00000000e+00]]
# Generating Cholesky Integrals
# frozen orbitals are 0
# local active orbitals are [0 1 2 3 4 5]
# local active space size 6
# loc_eris shape: (36, 36)
# chol shape: (15, 36)
# Finished calculating Cholesky integrals

# Size of the correlation space:
# Number of electrons: (3, 3)
# Number of basis functions: 6
# Number of Cholesky vectors: 15

# Number of MPI ranks: 1
#
# norb: 6
# nelec: (3, 3)
#
# n_eql: 2
# n_ene_blocks: 1
# n_sr_blocks: 10
# n_blocks: 10
# n_wa

In [5]:
print('lno-ccsd-afqmc ecorr: ',sum(block_orb_energy))
print(mf.e_tot + sum(block_orb_energy))
print('lno-ccsd ecorr: ',sum(ecorr_ccsd))
print(mf.e_tot + sum(ecorr_ccsd))

lno-ccsd-afqmc ecorr:  0.10410840464857202
-3.082256224425215
lno-ccsd ecorr:  -0.08361801384798825
-3.269982642921775
